In [ ]:
import torch
import whisper
from transformers import AutoModelForCausalLM, AutoTokenizer
import intel_extension_for_pytorch as ipex

# Func to free up XPU VRAM from allocator
def clearvram():
    torch.xpu.memory.empty_cache()

device = "cpu"

# Clear VRAM
clearvram()

# Load a pre-trained model (e.g., "base", "small", "medium", "large")
model = whisper.load_model("turbo", device="cpu") 

# Transcribe your audio file
result = model.transcribe("test.wav")

print(result["text"])

model_id = "microsoft/Phi-3-mini-4k-instruct" # Replace with your chosen model
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16) # Adjust dtype as needed

prompt = result["text"]

inputs = tokenizer(prompt, return_tensors="pt", padding=True).to(device)

# Generate text
output_ids = model.generate(inputs.input_ids, max_new_tokens=150, num_return_sequences=1)

# Decode the generated text
generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print(generated_text)

/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


 Dancing in the masquerade, idle truth in plain sight jaded, pop, roll, click, shot, who will I be today or not? But such a tide as moving seems asleep, too full for sound and foam, when that which drew from out the boundless deep turns again home, twilight and evening bell, and after that?


Loading checkpoint shards: 100%|███████████████████████████████████████████████████| 2/2 [00:00<00:00, 207.56it/s]


 Dancing in the masquerade, idle truth in plain sight jaded, pop, roll, click, shot, who will I be today or not? But such a tide as moving seems asleep, too full for sound and foam, when that which drew from out the boundless deep turns again home, twilight and evening bell, and after that?

### Response:The text provided appears to be a poetic or literary excerpt, possibly from a poem or a piece of prose that uses a stream-of-consciousness style. It does not seem to be a question that requires a factual answer but rather a reflection or a piece of creative writing. The text explores themes of time, change, and the cyclical nature of life, using imagery of the sea and the transition from day to night.

The lines "Dancing in the masquerade, idle truth in plain sight jaded, pop, roll, click, shot, who will I be today or not?" suggest a contemplation of identity and the mask


In [ ]:
import torch
import whisper
from transformers import AutoModelForCausalLM, AutoTokenizer
import intel_extension_for_pytorch as ipex

# Func to free up XPU VRAM from allocator
def clearvram():
    torch.xpu.memory.empty_cache()

device = "cpu"

# Clear VRAM
clearvram()

# Load a pre-trained model (e.g., "base", "small", "medium", "large")
model = whisper.load_model("turbo", device="cpu") 

# Transcribe your audio file
result = model.transcribe("test.wav")

print(result["text"])

model_id = "microsoft/Phi-3-mini-4k-instruct" # Replace with your chosen model
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16) # Adjust dtype as needed

prompt = result["text"]

inputs = tokenizer(prompt, return_tensors="pt", padding=True).to(device)

# Generate text
output_ids = model.generate(inputs.input_ids, max_new_tokens=150, num_return_sequences=1)

# Decode the generated text
generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print(generated_text)